In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import HistGradientBoostingRegressor
import statistics as sts
from sliceline.slicefinder import Slicefinder
import optbinning

(CVXPY) Nov 27 06:18:19 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Nov 27 06:18:19 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


**Reproducibility Notes**

The PyPi `sliceline` package requires Python 3.7~3.10.0, which is not the most up-to-date python version. Creating a virtual environment with python 3.9 should work. 

In [3]:
df = pd.read_csv('data/housing.csv')
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
# Really stupid statistics
print('longitude', min(set(df['longitude'])), max(set(df['longitude'])))
print('latitude', min(set(df['latitude'])), max(set(df['latitude'])))
print('median age', set(df['housing_median_age']))
print('total rooms', min(set(df['total_rooms'])), max(set(df['total_rooms'])))
print('total bedrooms', min(set(df['total_bedrooms'])), max(set(df['total_bedrooms'])))
print('population', min(set(df['population'])), max(set(df['population'])))
print('households', min(set(df['households'])), max(set(df['households'])))
print('median income', min(set(df['median_income'])), max(set(df['median_income'])))
print('median house value', min(set(df['median_house_value'])), max(set(df['median_house_value'])))
print('ocean proximity', set(df['ocean_proximity']))

longitude -124.35 -114.31
latitude 32.54 41.95
median age {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0}
total rooms 2.0 39320.0
total bedrooms nan nan
population 3.0 35682.0
households 1.0 6082.0
median income 0.4999 15.0001
median house value 14999.0 500001.0
ocean proximity {'INLAND', 'ISLAND', '<1H OCEAN', 'NEAR OCEAN', 'NEAR BAY'}


In [5]:
# Shuffle DF to simulate randomness
df = df.sample(frac=1)
# Split data into 5 splits (train, test, 3 additional DS)
num_splits = 5
splits = [df[i::num_splits] for i in range(num_splits)]
for i, split in enumerate(splits):
    split.to_csv('data/housing' + str(i) + '.csv', index=False)

In [6]:
train_df = pd.read_csv('data/housing0.csv')
train_y = train_df['median_house_value']
train_x = train_df.drop('median_house_value', axis=1)
train_x = pd.get_dummies(train_x, columns=['ocean_proximity'])

In [7]:
train_x

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-118.18,33.98,38.0,1477.0,374.0,1514.0,408.0,2.5703,True,False,False,False,False
1,-117.90,34.14,29.0,2240.0,457.0,1187.0,407.0,3.8365,True,False,False,False,False
2,-121.11,38.04,32.0,1083.0,188.0,471.0,178.0,2.9241,False,True,False,False,False
3,-117.66,34.07,33.0,2081.0,409.0,1008.0,375.0,2.5870,False,True,False,False,False
4,-122.30,37.87,10.0,503.0,118.0,228.0,100.0,2.1705,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,-118.12,33.84,37.0,2143.0,382.0,1047.0,377.0,4.4423,True,False,False,False,False
4124,-118.19,33.79,37.0,1834.0,551.0,1967.0,476.0,2.1370,False,False,False,False,True
4125,-115.64,32.80,23.0,1228.0,235.0,569.0,235.0,3.1667,False,True,False,False,False
4126,-121.97,37.25,32.0,2892.0,496.0,1193.0,492.0,6.1310,True,False,False,False,False


In [8]:
model = HistGradientBoostingRegressor(random_state=42)
model.fit(train_x, train_y)

HistGradientBoostingRegressor(random_state=42)

In [9]:
test_df = pd.read_csv('data/housing1.csv')
test_y = train_df['median_house_value']
test_x = train_df.drop('median_house_value', axis=1)
test_x = pd.get_dummies(test_x, columns=['ocean_proximity'])

In [10]:
y_pred = model.predict(test_x)
y_pred

array([148218.77327051, 206516.33347289, 148271.09578323, ...,
        85044.90498899, 381359.09255035,  73533.57813645])

In [11]:
training_errors = (test_y - y_pred)**2
training_errors

0       9.230189e+08
1       4.980187e+08
2       1.538907e+09
3       2.368607e+08
4       5.180291e+08
            ...     
4123    4.477730e+07
4124    2.043201e+08
4125    1.596410e+09
4126    1.838490e+08
4127    6.419018e+06
Name: median_house_value, Length: 4128, dtype: float64

In [12]:
mean_training_error = sts.mean(training_errors)
math.sqrt(mean_training_error)

34187.496552156124

In [13]:
optimal_binner = optbinning.ContinuousOptimalBinning(max_n_bins=5)

X_trans = pd.DataFrame(np.array(
    [
        optimal_binner.fit_transform(train_x[col], training_errors, metric="bins") for col in train_x.columns
    ]
).T, columns=train_x.columns)

In [14]:
X_trans

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,"[-122.44, -117.92)","[33.81, 37.90)","[33.50, 49.50)","[1388.50, 1520.50)","[135.50, 542.50)","[676.50, 2275.50)","[384.50, 443.50)","[2.27, 4.78)","[0.50, inf)","(-inf, 0.50)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
1,"[-117.92, -117.38)","[33.81, 37.90)","[28.50, 31.50)","[1520.50, inf)","[135.50, 542.50)","[676.50, 2275.50)","[384.50, 443.50)","[2.27, 4.78)","[0.50, inf)","(-inf, 0.50)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
2,"[-122.44, -117.92)","[37.90, 38.82)","[31.50, 33.50)","[966.50, 1388.50)","[135.50, 542.50)","[345.00, 676.50)","[120.50, 282.50)","[2.27, 4.78)","(-inf, 0.50)","[0.50, inf)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
3,"[-117.92, -117.38)","[33.81, 37.90)","[31.50, 33.50)","[1520.50, inf)","[135.50, 542.50)","[676.50, 2275.50)","[282.50, 384.50)","[2.27, 4.78)","(-inf, 0.50)","[0.50, inf)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
4,"[-122.44, -117.92)","[33.81, 37.90)","(-inf, 28.50)","(-inf, 589.50)","(-inf, 135.50)","(-inf, 345.00)","(-inf, 120.50)","[1.99, 2.27)","(-inf, 0.50)","(-inf, 0.50)","(-inf, inf)","[0.50, inf)","(-inf, 0.50)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,"[-122.44, -117.92)","[33.81, 37.90)","[33.50, 49.50)","[1520.50, inf)","[135.50, 542.50)","[676.50, 2275.50)","[282.50, 384.50)","[2.27, 4.78)","[0.50, inf)","(-inf, 0.50)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
4124,"[-122.44, -117.92)","[33.70, 33.81)","[33.50, 49.50)","[1520.50, inf)","[542.50, 592.50)","[676.50, 2275.50)","[443.50, inf)","[1.99, 2.27)","(-inf, 0.50)","(-inf, 0.50)","(-inf, inf)","(-inf, 0.50)","[0.50, inf)"
4125,"[-117.17, inf)","(-inf, 33.70)","(-inf, 28.50)","[966.50, 1388.50)","[135.50, 542.50)","[345.00, 676.50)","[120.50, 282.50)","[2.27, 4.78)","(-inf, 0.50)","[0.50, inf)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"
4126,"[-122.44, -117.92)","[33.81, 37.90)","[31.50, 33.50)","[1520.50, inf)","[135.50, 542.50)","[676.50, 2275.50)","[443.50, inf)","[4.78, inf)","[0.50, inf)","(-inf, 0.50)","(-inf, inf)","(-inf, 0.50)","(-inf, 0.50)"


In [25]:
# fitting sliceline
sf = Slicefinder(
    alpha = 0.9,
    k = 3,
    max_l = 4,
    min_sup = 0,
    verbose = True
)
sf.fit(X_trans, training_errors)

/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:457: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  and array.dtypes.apply(is_sparse).any()
/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:294: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:303: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
DEBUG:sliceline.slicefinder:Dropping 0/50 features below min_sup = 0.
DEBUG:sliceline.slicefinder:Initial top-K: count=2, max=0.132190, min=0.013215
/home/jwc/miniconda3/envs/sl

Slicefinder(alpha=0.9, k=3, min_sup=0)

In [26]:
sf.top_slices_

array([[None, None, '(-inf, 28.50)', None, None, None, None, None, None,
        '(-inf, 0.50)', None, None, None],
       [None, None, '(-inf, 28.50)', None, None, None, None, None, None,
        '(-inf, 0.50)', '(-inf, inf)', None, None],
       [None, None, None, None, None, None, None, None, None,
        '(-inf, 0.50)', None, None, None],
       [None, None, None, None, None, None, None, None, None,
        '(-inf, 0.50)', '(-inf, inf)', None, None]], dtype=object)

In [27]:
pd.DataFrame(sf.top_slices_, columns=sf.feature_names_in_, index=sf.get_feature_names_out())

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
slice_0,None,None,"(-inf, 28.50)",None,None,None,None,None,None,"(-inf, 0.50)",None,None,None
slice_1,None,None,"(-inf, 28.50)",None,None,None,None,None,None,"(-inf, 0.50)","(-inf, inf)",None,None
slice_2,None,None,None,None,None,None,None,None,None,"(-inf, 0.50)",None,None,None
slice_3,None,None,None,None,None,None,None,None,None,"(-inf, 0.50)","(-inf, inf)",None,None
